# Random Forest From Scratch

## CS235: Data Mining Techniques
### Dan O'Connor

In [1]:
#necessary imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#import and changr from dataframes to arrays for algo compatability #use arrays instead of pd.series, dfs
X_train_pca = pd.read_csv('X_train_pca.csv').to_numpy()
X_test_pca = pd.read_csv('X_test_pca.csv').to_numpy()
y_train = pd.read_csv('y_train.csv').to_numpy()
y_test = pd.read_csv('y_test.csv').to_numpy()

In [3]:
#confirm
type(X_train_pca), type(y_train)

(numpy.ndarray, numpy.ndarray)

## Random Forest Classifier

Here I will explain the idea behind a decision tree and my coding approach. 
I broke up creating the random forest classifier into three main sections. To make code run smoother and interact easier I decided to do object oriented by creating classes. A random forest is combination of multiple decision trees which are themselves a combination of multiple decision nodes. I took a bottom up approach and begin with nodes, then decision trees, then finally random forest. Below I will give description of the classes, their methods, and how they interact with each other to create a random forest classifier.



##### Node Class
This class contains no methods.
    
For each point in the decision tree where a decision is made, this class is utilized. This class has no methods, but rather is useful to store information about the feature being considered for the split, the threshold for splitting data, and any information gain from making that split. Each node can have a left and right child, representing the branches of decisions. When a node doesn't need to make a decision, it becomes a 'leaf node' holding a prediction value instead.

##### Decision Tree Classifier Class
1. Initialization(`__init__`):

- Sets up the maximum depth of the tree (max_depth), default is None
- Initializes the tree's root node as None

2. Fitting the Model (`fit`):

- Starting point for the decision tree
- Initializes the root by calling the `build_tree` method with the training data
- The `build_tree` method is called with a starting depth of 0 and set to the root node

3. Building the Tree (`build_tree`):

- First checks if the stopping conditions are met, such as reaching the maximum depth or arriving at a leaf node (a node with no further splits). If so, it returns a leaf node.
- If the stopping conditions are not met, it finds the best split using  `find_best_split` 
    - `find_best_split`  iteratively checks each feature and its possible threshold values to determine the most informative split, using the `information_gain`
        - `information_gain` calculates info gain of potential split using weighted children, which measures the purity of a split using `gini`
            - `gini` is used to calculate the gini impurity of a given subset of data
- Once the best split is found, `build_tree` creates a new node with left and right branches, corresponding to the split data
- Recursive process, building out the whole tree

4. Predicting Output (`predict`):

- Make predictions on unseen data (test set)
- Traverses the tree for each data point in the input X, following the path determined by the splits (based on feature values) until it reaches a leaf node
- Prediction for each data point is the value of the leaf node reached at the end of this path, determined by `calc_leaf_value`
    - `calc_leaf_value`: returns the most common class within the leaf node


##### Random Forest  Classifier Class

1. Initialization (`__init__`):

- Sets the number of decision trees in the forest (n_estimators), default is 100
- Defines the maximum depth for each tree (max_depth), default is None
- Initializes an empty list to store the trees
- max_features is set to None inititially


2. Fitting the Model (`fit`):

- Initializes the trees list as empty at the start of fitting
- Determines the number of features to consider when looking for the best split (max_features) as the square root of the total number of features
- For each of the n_estimators:
    - Creates a new instance of `DecisionTreeClassifier`
    - Generates a bootstrap sample of the training data using the bootstrap_sample method
    - Fits the decision tree to this bootstrap sample
    - Appends the fitted tree to the trees list
           
3. Bootstrap Sampling (`bootstrap_sample`):

- Randomly samples the indices of the training data (X) with replacement
- The size of the bootstrap sample is equal to the size of the original training data.

4. Predicting Output (`predict`):

- For each data point in the input X, each tree in the forest makes a prediction.
- Returns the majority vote class


### Node Class

In [4]:
class Node:
    '''
    Node in DT.
    
    Feat_idx: feature index of the node
    Threshold: the threshold value of split for that feature
    left: left child
    right: right child
    leaf_value: the leaf value
    info_gain: the information gain at the node
    
    '''
    
    def __init__(self, feat_idx=None, threshold=None, left=None, right=None, leaf_value=None, info_gain=None):
        #for decision nodes
        self.feature_idx = feat_idx
        self.threshold = threshold
        self.left = left 
        self.right = right
        self.info_gain = info_gain
        
        #for leaf node
        self.leaf_value = leaf_value

### Decision Tree Class


In [46]:

class DecisionTreeClassifier:
    
    def __init__(self, max_depth=None, max_features=None):
        #initialze max depth
        self.max_depth = max_depth
        self.max_features = max_features
        #initializing the root as `None`
        self.root = None
        
    def information_gain(self, y, l_y, r_y):
        ''' 
        function to compute information gain, using the class values (y)
        Determines weighted purity of split at node
        
        '''
        #weighted child weights
        weight_l = len(l_y) / len(y)
        weight_r = len(r_y) / len(y)
        
        #Gini information gain
        info_gain = self.gini(y) - (weight_l*self.gini(l_y) + weight_r*self.gini(r_y))    
        return info_gain
    
    
    def gini(self, y):
        #gini formula, where m is the number of classes. In this case, only binary classification but kept formula general.
        m = len(y)
        return 1 - sum(((np.sum(y == c))/ m) ** 2 for c in np.unique(y))
    
    def calc_leaf_value(self, y):
        #returns most frequent label on leaf
        return np.argmax(np.bincount(y))
    
    def find_best_split(self, X, y):

        #brute force finding the best splits, going through all the features and possible thresholds
        best_split = {} #dictionary to store the splits
        max_info = -np.inf #maximum possible, will act as placeholder for first iteration
        
        
        n_feats = X.shape[1]
        
        if self.max_features is not None:
            features = np.random.choice(range(n_feats), size=self.max_features, replace=False)
        else:
            features = range(n_feats)
        #iterate through features
        for feat_idx in features:
            feat_values = X[:,feat_idx] #all of the values for that feature 
            poss_thresh = np.unique(feat_values) #all possible thresholds (all unique values of that feature)
            
            #iterate through thresholds to find information gain (how good of split)
            for threshold in poss_thresh:
                
                #split the data, using feature index and threshold value
                left_x, right_x, left_y, right_y = self.split_data(X, y, feat_idx, threshold)
                
                #ensuring there is data on each child
                if (len(left_y) > 0) and (len(right_y)> 0):
                    
                    #              
                    current_info = self.information_gain(y, left_y, right_y)
                    
                    #if the current info of iteration is greater, update dictionary (better split)
                    if current_info > max_info:
                        best_split['feat_idx'] = feat_idx
                        best_split['threshold'] = threshold
                        best_split['right_X'] = right_x
                        best_split['left_X'] = left_x
                        best_split['right_y'] = right_y
                        best_split['left_y'] = left_y
                        best_split['info_gain'] = current_info
                        max_info = current_info
                
        return best_split
                
    def split_data(self, X, y, feat_idx, threshold):
        '''
        Splits the data
        
        '''
        #split the data by threshold
        left_mask = X[:, feat_idx] <= threshold
        right_mask = X[:, feat_idx] > threshold

        #X data
        left_data_X = X[left_mask]
        right_data_X = X[right_mask]
        
        #proper structure w/ ravel for y data
        left_data_y = np.ravel(y[left_mask]) 
        right_data_y = np.ravel(y[right_mask]) 

        return left_data_X, right_data_X, left_data_y, right_data_y

    def build_tree(self, X, y, current_depth=0):

        #Stopping condition. If the depth reaches max depth (when specified) OR a child is pure, return the leaf node.
        if len(np.unique(y)) == 1 or (self.max_depth is not None and current_depth >= self.max_depth):
            return Node(leaf_value=self.calc_leaf_value(y))
        
        #find the best split 
        best_split = self.find_best_split(X, y)
        
        #if the dictionary exists and information gain is greater than 0, continue recursively
        if best_split and best_split['info_gain'] > 0:
            
            #recursively build l and r branches of tree
            left_branch = self.build_tree(best_split['left_X'], best_split['left_y'], current_depth + 1)
            right_branch = self.build_tree(best_split['right_X'], best_split['right_y'], current_depth + 1)

            #return node w/ branches attached to it
            return Node(
                feat_idx=best_split['feat_idx'], 
                threshold=best_split['threshold'], 
                left=left_branch, 
                right=right_branch,
                info_gain=best_split['info_gain']
            )

        #if no split is found, return leaf node and calculate majority.
        return Node(leaf_value=self.calc_leaf_value(y))
    
    def predict(self, X):
    
        
        #empty list to store predictions
        predictions = []
        
        #iterate thru each data point
        for inputs in X:
            #begin form root
            node = self.root
            
            #Traverse until leaf is reach
            while node.left is not None or node.right is not None:
                
                #decide left or right based on node threshold
                if inputs[node.feature_idx] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
                    
            #once node is reached, append leaf value to pred        
            predictions.append(node.leaf_value)
        return predictions

    def fit(self, X, y):
        #fit the DT
        self.root = self.build_tree(X, y, current_depth=0)
        

Checking if it functions as expected on the iris dataset

In [27]:
#lets check this on a known dataset where 100% acc is feasible
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

#dataset
data = load_iris()
X = data.data
y = data.target

#convert to binary classification
y_binary = np.where(y == 0, 0, 1)

#train samples
X_sample = X[:100]
y_sample = y_binary[:100]

#dt defined above
dt = RandomForestClassifier()
dt.fit(X_sample, y_sample)

#preds
predictions = dt.predict(X_sample)
accuracy = accuracy_score(y_sample, predictions)
print("Accuracy:", accuracy * 100)



Accuracy: 100.0


### Random Forest Class

In [47]:
class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = None
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        
        #sqrt max feaets
        self.max_features = int(np.sqrt(X.shape[1]))
        
        for _ in range(self.n_estimators):
            tree = DecisionTreeClassifier(max_depth=self.max_depth, max_features=self.max_features)
            X_sample, y_sample = self.bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def bootstrap_sample(self, X, y):
        #num rows
        n_samples = X.shape[0]

        #random sample of indices with replacement. Size is equal to size of original
        indices = np.random.choice(n_samples, size=n_samples, replace=True)
        
        return X[indices], y[indices]

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)  #swap axes 
        majority_votes = np.array([np.argmax(np.bincount(tree_pred)) for tree_pred in tree_preds])
        
        return majority_votes


## Test on Dataset

In [35]:
#going to sample the data to do a comprehensive search with much smaller size to reduce computation time
from sklearn.model_selection import train_test_split


X_sample, _, y_sample, _ = train_test_split( X_train_pca, y_train, stratify=y_train, test_size=0.8, random_state=42)

In [30]:
X_sample.shape, y_sample.shape

((4800, 15), (4800, 1))

In [31]:

# SMOTE to 'balance' training  dataset
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_pca, y_train)


In [32]:
X_train_smote.shape, y_train_smote.shape

((37382, 15), (37382,))

In [33]:
X_sample_SMOTE, _, y_sample_SMOTE, _ = train_test_split( X_train_smote, y_train_smote, stratify=y_train_smote,\
                                                        test_size=0.8, random_state=42)

In [34]:
X_sample_SMOTE.shape, y_sample_SMOTE.shape

((4485, 15), (4485,))

From previous work on finding best parameters with SK-learn RF. Will impliment what I can with my model. Can control max_depth and n_estimators

Optimized RF: Best parameters: {'criterion': 'gini', 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}

SMOTE optimized RF: Best parameters: {'criterion': 'gini', 'max_depth': 12, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 110}

In [60]:
from sklearn.metrics import classification_report

rf = RandomForestClassifier(max_depth=6, n_estimators=100)
rf.fit(X_sample, y_sample)
y_pred= rf.predict(X_test_pca)
class_report = classification_report(y_test, y_pred)
print(class_report)


              precision    recall  f1-score   support

           0       0.80      0.95      0.87      4673
           1       0.50      0.19      0.28      1327

    accuracy                           0.78      6000
   macro avg       0.65      0.57      0.57      6000
weighted avg       0.74      0.78      0.74      6000



In [61]:
rf_smote = RandomForestClassifier(max_depth=12, n_estimators=110)
rf_smote.fit(X_sample_SMOTE, y_sample_SMOTE)
y_pred_SMOTE = rf_smote.predict(X_test_pca)
class_report_SMOTE = classification_report(y_test,y_pred_SMOTE)
print(class_report_SMOTE)

              precision    recall  f1-score   support

           0       0.88      0.80      0.84      4673
           1       0.46      0.60      0.52      1327

    accuracy                           0.76      6000
   macro avg       0.67      0.70      0.68      6000
weighted avg       0.78      0.76      0.77      6000

